---
format : html
echo : False
---

# Analyse PDM

**Présentation des résultats de l'examen du MELS pour la PDM.**



In [ ]:
from datetime import datetime
from IPython.display import display, Markdown

current_datetime = datetime.now().strftime('%d/%m/%Y %H:%M')
display(Markdown(f"Rapport produit par Cédric Lejeune, CP Mathématiques Secondaire CSSLaurentides le {current_datetime}"))

**Introduction**

Analyse des résultats des différentes épreuves en math CST secondaire 4 de la PDM de l'année à la lumière des résultats de l'examen du MELS.

Objectif : Réguler les différentes évaluations de l'année.

Notes sur les données : Seul les élèves ayant fait l'examen du MELS ont été retenus dans les analyses.

In [1]:
import pandas as pd

# Try two possible file paths for the data
try:
    file_path = '/content/drive/MyDrive/Travail/AnalyseSec4PDM/data_sansnoms.csv'
    df = pd.read_csv(file_path, encoding='cp1252', sep=';')
except (FileNotFoundError, OSError):
    file_path = 'data_sansnoms.csv'
    df = pd.read_csv(file_path, encoding='cp1252', sep=';')

# ...existing code for df_scores and cleaning...
df_scores = df[['Groupe', 'C2 Final École', 'C2 Final Modérée', 'Examen MELS']].copy()

# Identify and replace non-numeric values with NaN
df_scores['C2 Final École'] = df_scores['C2 Final École'].replace(['#VALEUR!', 'NE'], pd.NA)
df_scores['C2 Final Modérée'] = df_scores['C2 Final Modérée'].replace(['#VALEUR!', 'NE'], pd.NA)
df_scores['Examen MELS'] = df_scores['Examen MELS'].replace(['#VALEUR!', 'NE'], pd.NA)

# Convert columns to numeric, coercing errors to NaN
df_scores['C2 Final École'] = pd.to_numeric(df_scores['C2 Final École'], errors='coerce')
df_scores['C2 Final Modérée'] = pd.to_numeric(df_scores['C2 Final Modérée'], errors='coerce')
df_scores['Examen MELS'] = pd.to_numeric(df_scores['Examen MELS'], errors='coerce')

# Drop rows with NaN values in the score columns
df_scores.dropna(subset=['C2 Final École', 'C2 Final Modérée', 'Examen MELS'], inplace=True)

df_melted = df_scores.melt(id_vars=['Groupe'], var_name='Score Type', value_name='Score')


In [ ]:
# Calcul des pourcentages de passage et de vulnérabilité
scores = df_scores['Examen MELS'].dropna()

# Pourcentage d'élèves ayant obtenu la note de passage (> 60)
pourcentage_passage = round((scores[scores > 60].count() / scores.count()),4) * 100 if scores.count() > 0 else 0

# Pourcentage d'élèves hors de la zone de vulnérabilité (> 75)
pourcentage_vulnerabilite = round((scores[scores > 75].count() / scores.count()),4) * 100 if scores.count() > 0 else 0


In [ ]:
# Distribution des résultats de l'examen du MELS par groupe


# Create and display the dynamic Markdown
display(Markdown(f"{pourcentage_vulnerabilite}% des élèves se situent hors de la zone de vulnérabilité."))
display(Markdown(f"{pourcentage_passage}% des élèves se ont obtenus la note de passage."))


In [2]:
# Histogramme des résultats de l'examn du MELS

import plotly.graph_objects as go
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
import plotly # Import the top-level plotly module


# Ensure 'Score' and 'Groupe' are valid and drop NaNs
df_melted_filtered = df_melted.dropna(subset=['Score', 'Groupe']).copy()

# Filter to include only 'Examen MELS' scores
df_melted_filtered = df_melted_filtered[df_melted_filtered['Score Type'] == 'Examen MELS'].copy()


# Create traces for each group's histogram and density curve, and the overall histogram and density curve
traces = []

# Define bin settings
bin_settings = dict(start=0, end=100, size=5) # Bins from 0 to 100 with size 5 (20 bins)

# Add overall histogram trace
overall_hist = go.Histogram(x=df_melted_filtered['Score'],
                           xbins=bin_settings,
                           name='All Groups (Histogram)',
                           marker_color='gray',
                           opacity=1.0, # Full opacity for selected view
                           marker_line=dict(width=1, color='black'),
                           visible=True) # Visible by default
traces.append(overall_hist)


# Add histogram traces for each group
colors = plotly.colors.qualitative.Plotly # Use a color sequence
groups = sorted(df_melted_filtered['Groupe'].unique()) # Sort groups alphabetically
for i, group in enumerate(groups):
    group_data = df_melted_filtered[df_melted_filtered['Groupe'] == group]
    color = colors[i % len(colors)]

    # Add histogram trace for the group
    traces.append(go.Histogram(x=group_data['Score'],
                               xbins=bin_settings,
                               name=f'Groupe {group} (Histogram)',
                               marker_color=color,
                               opacity=1.0, # Full opacity for selected view
                               marker_line=dict(width=1, color='black'),
                               visible=False)) # Hidden by default

# Add vertical line trace (initially at 60)
initial_line_x = 60

# Create the figure
fig = go.Figure(data=traces)
all_group = True # Set all_group to True to display dropdown menu
# Create buttons for updatemenus
buttons = [
    dict(label='All Groups',
         method='update',
         # Find the indices of the 'All Groups' histogram and density traces
         args=[{'visible': [t.name == 'All Groups (Histogram)' for t in traces]}, # Show All Groups histogram, hide others
               {'title': 'Distribution des Scores (Tous Groupes)'}]) # Update title
]

# Add buttons for each group if all_group is True
if all_group:
    # Need to ensure the order of visibility corresponds to the sorted groups
    group_names = sorted(df_melted_filtered['Groupe'].unique()) # Get sorted group names again for indexing
    for i, group in enumerate(group_names):
        group_traces_visibility = [False] * len(traces) # Start with all hidden

        # Find the index of the current group's histogram trace
        hist_trace_index = traces.index(next(t for t in traces if t.name == f'Groupe {group} (Histogram)'))
        group_traces_visibility[hist_trace_index] = True # Set the current group's histogram trace to visible

        buttons.append(dict(label=f'Groupe {group}',
                            method='update',
                            args=[{'visible': group_traces_visibility},
                                  {'title': f'Distribution des résultat de l\'examen du MELS (Groupe {group})'}])) # Update title

# Calculate initial percentages and counts
total_scores = len(df_melted_filtered)
scores_less_than_initial = df_melted_filtered[df_melted_filtered['Score'] < initial_line_x].shape[0]
scores_greater_than_initial = df_melted_filtered[df_melted_filtered['Score'] >= initial_line_x].shape[0]
percentage_less_than_initial = (scores_less_than_initial / total_scores) * 100 if total_scores > 0 else 0
percentage_greater_than_initial = (scores_greater_than_initial / total_scores) * 100 if total_scores > 0 else 0


# Add annotations for percentages and counts
annotations = [
    dict(
        text=f'{percentage_less_than_initial:.2f}% ({scores_less_than_initial}) < {initial_line_x}',
        x=initial_line_x, # Set x position to the threshold value
        y=1.05, # Adjust y position relative to the top of the plot
        xref='x', # Use x-axis as reference
        yref='paper', # Use paper as reference for y (relative to the plot)
        showarrow=False,
        font=dict(color="blue", size=12),
        xanchor='right' # Anchor to the right of the text
    ),
    dict(
        text=f'{percentage_greater_than_initial:.2f}% ({scores_greater_than_initial}) >= {initial_line_x}',
        x=initial_line_x, # Set x position to the threshold value
        y=1.05, # Adjust y position relative to the top of the plot
        xref='x', # Use x-axis as reference
        yref='paper', # Use paper as reference for y (relative to the plot)
        showarrow=False,
        font=dict(color="blue", size=12),
        xanchor='left' # Anchor to the left of the text
    )
]


# Add a slider to the layout
steps = []
for i in range(0, 101): # Slider from 0 to 100

    # Calculate percentages and counts for the current threshold
    scores_less_than = df_melted_filtered[df_melted_filtered['Score'] < i].shape[0]
    scores_greater_than = df_melted_filtered[df_melted_filtered['Score'] >= i].shape[0]
    percentage_less_than = (scores_less_than / total_scores) * 100 if total_scores > 0 else 0
    percentage_greater_than = (scores_greater_than / total_scores) * 100 if total_scores > 0 else 0


    step = dict(
        method='relayout',
        args=[{
            'shapes': [{'x0': i, 'x1': i, 'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper'}], # Update vertical line position
            'annotations': [ # Update annotations
                dict(
                    text=f'{percentage_less_than:.2f}% ({scores_less_than}) < {i}',
                    x=i, # Set x position to the threshold value
                    y=1.05, # Adjust y position relative to the top of the plot
                    xref='x', # Use x-axis as reference
                    yref='paper', # Use paper as reference for y (relative to the plot)
                    showarrow=False,
                    font=dict(color="blue", size=12),
                    xanchor='right'
                ),
                dict(
                    text=f'{percentage_greater_than:.2f}% ({scores_greater_than}) >= {i}',
                    x=i, # Set x position to the threshold value
                    y=1.05, # Adjust y position relative to the top of the plot
                    xref='x', # Use x-axis as reference
                    yref='paper', # Use paper as reference for y (relative to the plot)
                    showarrow=False,
                    font=dict(color="blue", size=12),
                    xanchor='left'
                )
            ]
        }],
        label=str(i)
    )
    steps.append(step)

sliders = [dict(
    active=60, # Initial position at 60
    steps=steps,
    pad={"t": 50}, # Add padding at the top
    currentvalue={"prefix": "Threshold: "} # Add a prefix to the current value display
)]

# Create updatemenu based on all_group
updatemenus = []
if all_group:
    updatemenus=[
        dict(
            type='dropdown',
            direction='down',
            x=1.0, # Set x to 1 for right alignment
            y=1.15, # Keep y slightly above the plot for the dropdown
            xanchor='right', # Anchor the dropdown to the right
            yanchor='top', # Anchor the dropdown to the top
            showactive=True,
            buttons=buttons
        )
    ]


# Update layout with updatemenus, slider, and initial annotations
fig.update_layout(
    updatemenus=updatemenus,
    sliders=sliders,
    title_text='Distribution des Scores (Tous Groupes)', # Initial title
    xaxis_title='Score',
    yaxis_title='Nombre d\'occurrences', # Changed y-axis title to Number of occurrences
    xaxis=dict(range=[0, 100], tickmode='linear', tick0=0, dtick=10),
    # Remove fixed y-axis range to allow Plotly to autoscale
    # yaxis=dict(range=[0, max_density_estimate * 1.2]), # Removed
    template='plotly_white',
    font=dict(family="sans-serif", size=12, color="#000"),
    annotations=annotations # Add initial annotations to the layout
    )

# Add initial vertical line shape
fig.add_shape(type="line",
              x0=initial_line_x, y0=0, x1=initial_line_x, y1=1,
              xref='x', yref='paper',
              line=dict(color="blue", width=2, dash="dash"))


# Show the interactive plot
fig.show() # Removed renderer="colab"

ModuleNotFoundError: No module named 'plotly'

In [ ]:
# @title Default title text
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Recreate df_selected_columns and calculate the correlation matrix
columns_to_keep = [col for col in df.columns if 'C1' not in col and col != 'Groupe' and col != 'Fiche']
df_selected_columns = df[columns_to_keep].copy()

for col in df_selected_columns.columns:
    df_selected_columns[col] = pd.to_numeric(df_selected_columns[col], errors='coerce')

correlation_matrix = df_selected_columns.corr()

# Identify the columns from the second correlation matrix
columns_from_corr_matrix = correlation_matrix.columns.tolist()

# Identify columns to exclude
excluded_columns = ['C2 Final École', 'C2 Final Modérée', 'Modération', 'C2 Final Bulletin']

# Create a list of feature column names based on the exclusion criteria, keeping only those from the second correlation matrix
feature_columns_for_scatter = [col for col in columns_from_corr_matrix if col not in excluded_columns and col != 'Examen MELS']

# Create a scatter plot for the difference between Examen MELS and each feature vs. the feature
for col in feature_columns_for_scatter:
    plt.figure(figsize=(8, 6))

    # Select the relevant columns for the current plot and drop rows with NaN
    temp_df = df[[col, 'Examen MELS']].copy()
    for temp_col in temp_df.columns:
        temp_df[temp_col] = pd.to_numeric(temp_df[temp_col], errors='coerce')
    temp_df.dropna(inplace=True)

    # Calculate the difference
    if not temp_df.empty:
        temp_df['Difference'] = temp_df['Examen MELS'] - temp_df[col]

        sns.scatterplot(data=temp_df, x=col, y='Difference')
        plt.title(f'Nuage de Points de la Différence Examen MELS - {col} vs. {col}') # Translated title
        plt.xlabel(f'Score {col}') # Translated x-label
        plt.ylabel(f'Différence (Examen MELS - {col})') # Translated y-label

        # Set x and y axis limits
        plt.xlim(0, 100)
        plt.ylim(-100, 100) # Adjust y-limit as differences can be negative

        # Add horizontal line at y=0
        plt.axhline(0, color='gray', linestyle='--')

        # Add vertical line at x=60
        plt.axvline(60, color='gray', linestyle='--')


        plt.show()

In [ ]:
for col in feature_columns_for_scatter:
    # Select the relevant columns and handle non-numeric values
    temp_df = df[[col, 'Examen MELS']].copy()
    for temp_col in temp_df.columns:
        temp_df[temp_col] = pd.to_numeric(temp_df[temp_col], errors='coerce')
    temp_df.dropna(inplace=True)

    # Filter data points around (20, 90) - adjust tolerance as needed
    # Assuming 'around (20, 90)' means within a certain range of x=20 and y=90
    x_tolerance = 10  # Adjust the tolerance for the x-axis (feature score)
    y_tolerance = 10  # Adjust the tolerance for the y-axis (Examen MELS score)

    filtered_df = temp_df[
        (temp_df[col] >= 20 - x_tolerance) & (temp_df[col] <= 20 + x_tolerance) &
        (temp_df['Examen MELS'] >= 90 - y_tolerance) & (temp_df['Examen MELS'] <= 90 + y_tolerance)
    ].copy()


    if not filtered_df.empty:
        # Calculate the difference between 'Examen MELS' and the feature score
        filtered_df['Difference'] = filtered_df['Examen MELS'] - filtered_df[col]

        # Calculate the mean difference
        mean_diff = filtered_df['Difference'].mean()

        print(f"Mean difference for {col} vs. Examen MELS around (20, 90): {mean_diff:.2f}")
    else:
        print(f"No data points found for {col} vs. Examen MELS around (20, 90) with the current tolerance.")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

g = sns.FacetGrid(df_melted, row='Groupe', col='Score Type', height=3)
g.map(sns.kdeplot, 'Score', fill=True, bw_adjust=0.35)
g.add_legend(title='Groupe')
g.fig.suptitle('Distribution des Scores par Groupe et Type de Score (KDE)', y=1.02) # Translated title
g.set_axis_labels("Score", "Densité") # Translated axis labels
g.set(xlim=(0, 100)) # Set x-axis limits to 0 and 100
g.set_titles(template='{col_name} - Groupe {row_name}') # Set subplot titles

# Add x-axis ticks and labels
for ax in g.axes.flat:
    ax.set_xticks(np.arange(0, 101, 20)) # Set ticks every 20 units from 0 to 100
    ax.set_xticklabels(np.arange(0, 101, 20)) # Set labels for the ticks

plt.tight_layout()
plt.show()

In [ ]:
df_numeric = df.drop('Groupe', axis=1)
for col in df_numeric.columns:
    df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')

correlation_matrix = df_numeric.corr()

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matrice de Corrélation des Scores') # Translated title
plt.show()

**Reasoning**:
Select columns that do not contain 'C1' and are not 'Groupe', then display the head of the new dataframe.



In [ ]:
columns_to_keep = [col for col in df.columns if 'C1' not in col and col != 'Groupe' and col != 'Fiche']
df_selected_columns = df[columns_to_keep]
display(df_selected_columns.head())

**Reasoning**:
Convert all columns in the selected DataFrame to numeric, coercing errors to NaN.



In [ ]:
for col in df_selected_columns.columns:
    df_selected_columns[col] = pd.to_numeric(df_selected_columns[col], errors='coerce')

display(df_selected_columns.head())

## Calculate correlation

### Subtask:
Calculate the correlation matrix for the selected numeric columns.


**Reasoning**:
Calculate the pairwise correlation of columns in df_selected_columns and store the result in a variable named correlation_matrix.



In [ ]:
correlation_matrix = df_selected_columns.corr()
display(correlation_matrix)

## Visualize correlation

### Subtask:
Create a heatmap to visualize the correlation matrix.


**Reasoning**:
Create a heatmap of the correlation matrix to visualize the relationships between the numeric columns, including annotations and a title.



In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matrice de Corrélation des Scores Sélectionnés') # Translated title
plt.show()

In [ ]:
from scipy.stats import pearsonr
import pandas as pd

# Get column names from the correlation matrix
columns = correlation_matrix.columns

# Create an empty DataFrame to store p-values
p_values_matrix = pd.DataFrame(index=columns, columns=columns)

# Calculate p-values for each pair of columns
for col1 in columns:
    for col2 in columns:
        # Ensure both columns are in the original df_selected_columns and drop NaNs for the test
        temp_df = df_selected_columns[[col1, col2]].dropna()
        if len(temp_df) > 2: # Need at least 2 data points for correlation
            corr, p_value = pearsonr(temp_df[col1], temp_df[col2])
            # Ensure p_value is a scalar before assigning to the DataFrame
            p_values_matrix.loc[col1, col2] = p_value if isinstance(p_value, (int, float)) else None
        else:
            p_values_matrix.loc[col1, col2] = None # Not enough data

# Filter the p-value matrix to show only values less than 0.2
p_values_matrix_filtered = p_values_matrix.mask(p_values_matrix >= 0.2)

display(p_values_matrix_filtered)

In [ ]:
# Select the relevant columns and handle non-numeric values
df_comparison = df[['Examen MELS', 'C2 Final École']].copy()
df_comparison['Examen MELS'] = pd.to_numeric(df_comparison['Examen MELS'], errors='coerce')
df_comparison['C2 Final École'] = pd.to_numeric(df_comparison['C2 Final École'], errors='coerce')

# Drop rows with NaN in either column
df_comparison.dropna(inplace=True)

# Count rows where 'Examen MELS' is greater than 'C2 Final École'
count_greater_mels = df_comparison[df_comparison['Examen MELS'] > df_comparison['C2 Final École']].shape[0]

# Calculate the total number of relevant rows
total_rows = df_comparison.shape[0]

# Calculate the percentage
percentage_greater_mels = (count_greater_mels / total_rows) * 100 if total_rows > 0 else 0

print(count_greater_mels)
print(total_rows)

print(f"Percentage of rows with a greater score in Examen MELS than C2 Final École: {percentage_greater_mels:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Create a mask to hide the diagonal
diagonal_mask = np.eye(correlation_matrix.shape[0], dtype=bool)

# Create a masked correlation matrix
# Mask where p-value is 0.2 or greater OR absolute correlation is less than 0.4 OR it's the diagonal
mask_condition = (p_values_matrix >= 0.2) | (abs(correlation_matrix) < 0.4) | diagonal_mask
masked_correlation_matrix = correlation_matrix.mask(mask_condition)

plt.figure(figsize=(12, 10))
sns.heatmap(masked_correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", mask=masked_correlation_matrix.isnull())
plt.title('Matrice de Corrélation Significative (valeur p < 0.2 et |corrélation| >= 0.4, diagonale exclue)') # Translated title
plt.show()

In [ ]:
# Calculate the difference between 'Examen MELS' and 'C2 Final École'
df_comparison['Score Difference'] = df_comparison['Examen MELS'] - df_comparison['C2 Final École']

# Display the first few rows with the new column
display(df_comparison.head())

# Plot the distribution of the score difference
plt.figure(figsize=(10, 6))
min_diff = np.floor(df_comparison['Score Difference'].min() / 5) * 5
max_diff = np.ceil(df_comparison['Score Difference'].max() / 5) * 5
sns.histplot(data=df_comparison, x='Score Difference', kde=True, bins=np.arange(min_diff, max_diff + 5, 5))
plt.title('Distribution de la Différence entre les Scores de l\'Examen MELS et C2 Final École') # Translated title
plt.xlabel('Différence de Score (Examen MELS - C2 Final École)') # Translated x-label
plt.ylabel('Fréquence') # Translated y-label
plt.show()

In [ ]:
mean_difference = df_comparison['Score Difference'].mean()
print(f"The mean difference between Examen MELS and C2 Final École scores is: {mean_difference:.2f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_comparison, x='C2 Final École', y='Score Difference')

# Add horizontal line at y=0
plt.axhline(0, color='gray', linestyle='--')

# Add vertical line at x=60
plt.axvline(60, color='gray', linestyle='--')

plt.title('Différence de Score vs. C2 Final École') # Translated title
plt.xlabel('Score C2 Final École') # Translated x-label
plt.ylabel('Différence de Score (Examen MELS - C2 Final École)') # Translated y-label
plt.show()

In [ ]:
# Select the relevant columns and handle non-numeric values
df_evaluation_mels = df[['Evaluation mi-année', 'Examen MELS']].copy()
df_evaluation_mels['Evaluation mi-année'] = pd.to_numeric(df_evaluation_mels['Evaluation mi-année'], errors='coerce')
df_evaluation_mels['Examen MELS'] = pd.to_numeric(df_evaluation_mels['Examen MELS'], errors='coerce')

# Drop rows with NaN in either column
df_evaluation_mels.dropna(inplace=True)

# Create a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_evaluation_mels, x='Evaluation mi-année', y='Examen MELS')
plt.title('Nuage de Points de l\'Examen MELS vs. Évaluation mi-année') # Translated title
plt.xlabel('Score Évaluation mi-année') # Translated x-label
plt.ylabel('Score Examen MELS') # Translated y-label

# Add horizontal line at y=60
plt.axhline(60, color='gray', linestyle='--')

# Add vertical line at x=60
plt.axvline(60, color='gray', linestyle='--')


plt.show()

In [ ]:
# Identify columns to exclude
excluded_columns = ['Groupe', 'C2 Final École', 'C2 Final Modérée', 'Modération', 'C2 Final Bulletin', 'Note finale']

# Create a list of feature column names based on the exclusion criteria
feature_columns = [col for col in df.columns if col not in excluded_columns and 'C1' not in col]

# Create the features DataFrame X
X = df[feature_columns].copy()

# Create the target Series y
y = df['Examen MELS'].copy()


In [ ]:
rows_greater_than_95 = df_comparison[df_comparison['Score Difference'] > 95]
display(rows_greater_than_95)

In [ ]:
# !wget https://github.com/quarto-dev/quarto-cli/releases/download/v1.7.33/quarto-1.7.33-linux-amd64.deb
# !sudo dpkg -i quarto-1.7.33-linux-amd64.deb

In [ ]:
# !quarto render "/content/drive/MyDrive/Colab Notebooks/AnalyseModerationSec4.ipynb" \
#   --to html \
#   --embed-resources \
#   --include-in-header <(echo '<script src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>') > /dev/null 2>&1


In [ ]:
# !quarto render "AnalyseModerationSec4.ipynb" \
#   --to html \
#   --embed-resources \
#   --include-in-header <(echo '<script src="https://cdn.plot.ly/plotly-2.32.0.min.js"></script>') > /dev/null 2>&1
# ! firefox "AnalyseModerationSec4.html"



In [ ]:
# Install IPython if not already installed
# !pip install IPython